In [ ]:
import warnings
warnings.filterwarnings("ignore")
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col
from snowflake.snowpark import types as T
from snowflake.core import Root
from snowflake.cortex import Complete
session = get_active_session()

# Insurance Claims Analysis with Cortex Agents
In this notebook you will setup multiple Cortex Search and Cortex Analyst Services which will be used by Cortex Agents to answer user queries on unstructured and structured insurance data.
![text](https://github.com/michaelgorkow/snowflake_cortex_agents_demo/blob/main/resources/cortex_agents_notebook_small.png?raw=true)

# Setup the Cortex Search Service [Unstructured Data]

We have some PDF and markdown documents in our stage **DOCUMENTS** that we want insurance analysts to be able to ask questions about.  
To achieve this, we need to extract the contents of these files and make them searchable.

## Extracting Content from Document Files

### [`PARSE_DOCUMENT`](https://docs.snowflake.com/en/sql-reference/functions/parse_document-snowflake-cortex)  
This function returns the extracted content from a document on a Snowflake stage as an **OBJECT** that contains JSON-encoded objects as strings.  

It supports two types of extractions:  
- **Optical Character Recognition (OCR)**  
- **Layout Extraction**  

### [`SPLIT_TEXT_RECURSIVE_CHARACTER`](https://docs.snowflake.com/en/sql-reference/functions/split_text_recursive_character-snowflake-cortex)  
The `SPLIT_TEXT_RECURSIVE_CHARACTER` function splits a string into shorter strings recursively. It is useful for preprocessing text to be used with text embedding or search indexing functions.

In [ ]:
-- List documents in stage
SELECT * FROM DIRECTORY('@DOCUMENTS');

In [ ]:
-- Layout extraction for PDF and markdown documents
CREATE OR REPLACE TABLE RAW_TEXT AS
SELECT 
    RELATIVE_PATH,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@DOCUMENTS',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'} ):content
        ) AS EXTRACTED_LAYOUT 
FROM DIRECTORY ( '@DOCUMENTS' )
WHERE RELATIVE_PATH LIKE '%.pdf' 
   OR RELATIVE_PATH LIKE '%.md';

In [ ]:
-- Take a look at the extracted content
SELECT RELATIVE_PATH, SUBSTR(EXTRACTED_LAYOUT, 1, 500) FROM RAW_TEXT;

In [ ]:
-- Create a table with chunks of the extracted content
CREATE OR REPLACE TABLE TEXT_CHUNKS AS
SELECT
    RELATIVE_PATH,
    VALUE AS CHUNK,
    SEQ AS CHUNK_NUM
FROM RAW_TEXT,
LATERAL FLATTEN( INPUT => SPLIT_TEXT_RECURSIVE_CHARACTER (EXTRACTED_LAYOUT, 4000, 200, '.?!', 3) );

In [ ]:
-- Take a look at the text chunks
SELECT * FROM TEXT_CHUNKS ORDER BY RELATIVE_PATH, CHUNK_NUM LIMIT 3;

## Create Cortex Search Services

In this step we create two search services:
- `POLICY_DOCUMENTS_SEARCH`: For searching policy documentation
- `CLAIM_FORMS_SEARCH`: For searching claim forms and adjustor reports

A Snowflake Cortex Search service enables users to retrieve unstructured information by leveraging vector search technology. This implementation involves embedding content into vectors and then searching based on the semantic similarity between a query and the vectors.

In [ ]:
-- Create Cortex Search service for policy documents 
CREATE OR REPLACE SEARCH SERVICE POLICY_DOCUMENTS_SEARCH
EXTERNAL_ACCESS_INTEGRATION = CORTEX_ACCESS
COMPUTE_POOL = CORTEX_COMPUTE
MAX_KB = 50
CONTENT = (
    SELECT RELATIVE_PATH as SOURCE, CHUNK as CONTENT
    FROM   TEXT_CHUNKS
    WHERE  STARTSWITH(RELATIVE_PATH,'POLICY_DOCUMENTS')
);

-- Wait for the search service to be ready
CALL SYSTEM$WAIT(30);

In [ ]:
-- Create Cortex Search service for claim forms
CREATE OR REPLACE SEARCH SERVICE CLAIM_FORMS_SEARCH
EXTERNAL_ACCESS_INTEGRATION = CORTEX_ACCESS
COMPUTE_POOL = CORTEX_COMPUTE
MAX_KB = 50
CONTENT = (
    SELECT RELATIVE_PATH as SOURCE, CHUNK as CONTENT
    FROM   TEXT_CHUNKS
    WHERE  STARTSWITH(RELATIVE_PATH,'CLAIM_FORMS')
);

-- Wait for the search service to be ready
CALL SYSTEM$WAIT(30);

In [ ]:
-- List your search services
SHOW SEARCH SERVICES;

In [ ]:
-- Test search on policy documents (RAG example)
SELECT
    search,
    content,
    score,
    source,
    SUBSTR(source,POSITION("/", source)+1) as document
FROM TABLE(SEARCH(
    'POLICY_DOCUMENTS_SEARCH',
    'What is the deductible for comprehensive coverage?',
    LIMIT => 3
));


# Setup Tables and Search Services for Structured Data

Let's create some structured data to work with. We'll create tables for insurance claims data:

- `POLICYHOLDER_DATA`: information about policyholders
- `CLAIM_DATA`: information about insurance claims
- `COVERAGE_TYPES`: information about different coverage types

Then we'll create Search Services for each of these tables to enable semantic search on the structured data.

In [ ]:
-- Load the CSV data into tables
CREATE OR REPLACE TABLE CUSTOMER_DATA AS 
SELECT * FROM @SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER LIMIT 0;  -- Just creating schema

CREATE OR REPLACE TABLE CLAIM_DATA AS 
SELECT * FROM @SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.ORDERS LIMIT 0;  -- Just creating schema

CREATE OR REPLACE TABLE COVERAGE_TYPES AS 
SELECT * FROM @SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.PART LIMIT 0;  -- Just creating schema

-- Now load our actual data
COPY INTO CUSTOMER_DATA FROM @DATA_STAGE/customer_data.csv;
COPY INTO CLAIM_DATA FROM @DATA_STAGE/claim_data.csv;

-- Create our coverage types data
CREATE OR REPLACE TABLE COVERAGE_TYPES (
    COVERAGE_ID NUMBER,
    COVERAGE_TYPE VARCHAR,
    DESCRIPTION VARCHAR,
    BASE_PREMIUM NUMBER,
    DEDUCTIBLE_OPTIONS ARRAY
);

INSERT INTO COVERAGE_TYPES VALUES
(1, 'Comprehensive', 'Covers damage from non-collision incidents such as theft, vandalism, and natural disasters', 850, ARRAY_CONSTRUCT(250, 500, 1000)),
(2, 'Collision', 'Covers damage to your vehicle from a collision with another vehicle or object', 1200, ARRAY_CONSTRUCT(500, 1000, 2000)),
(3, 'Liability', 'Covers damage you cause to others and their property', 600, ARRAY_CONSTRUCT(0)),
(4, 'Personal Injury', 'Covers medical expenses for you and your passengers', 450, ARRAY_CONSTRUCT(250, 500)),
(5, 'Uninsured Motorist', 'Covers damages when the at-fault driver has no insurance', 350, ARRAY_CONSTRUCT(250, 500));

In [ ]:
-- Check the structured data
SELECT * FROM CUSTOMER_DATA LIMIT 5;

In [ ]:
-- Check the claims data
SELECT * FROM CLAIM_DATA LIMIT 5;

In [ ]:
-- Check the coverage types data
SELECT * FROM COVERAGE_TYPES;

## Create Cortex Search Services for Structured Data

The Cortex Search service for structured data serves the purpose of enabling dynamic literal retrieval for the Cortex Analyst service. For example, when a user asks "Show me claims for policyholder James Wilson", the Cortex Analyst service needs to translate "James Wilson" into a specific policy number. The search service helps in retrieving such literals from structured data.

In [ ]:
-- Create a Search Service for Coverage Types
CREATE OR REPLACE SEARCH SERVICE COVERAGE_TYPE_SEARCH
EXTERNAL_ACCESS_INTEGRATION = CORTEX_ACCESS
COMPUTE_POOL = CORTEX_COMPUTE
CONTENT = (
    SELECT 
        COVERAGE_TYPE || ': ' || DESCRIPTION as CONTENT,
        TO_VARIANT({
            'COVERAGE_ID': COVERAGE_ID, 
            'COVERAGE_TYPE': COVERAGE_TYPE,
            'BASE_PREMIUM': BASE_PREMIUM
        }) as ATTRIBUTES
    FROM   COVERAGE_TYPES
);

-- Wait for the search service to be ready
CALL SYSTEM$WAIT(30);

In [ ]:
-- Create a Search Service for Policyholder Information
CREATE OR REPLACE SEARCH SERVICE POLICYHOLDER_SEARCH
EXTERNAL_ACCESS_INTEGRATION = CORTEX_ACCESS
COMPUTE_POOL = CORTEX_COMPUTE
CONTENT = (
    SELECT 
        INSURED_EDUCATION_LEVEL || ' ' || INSURED_OCCUPATION || ' (' || POLICY_NUMBER || ')' as CONTENT,
        TO_VARIANT({
            'POLICY_NUMBER': POLICY_NUMBER, 
            'AGE': AGE,
            'INSURED_SEX': INSURED_SEX,
            'POLICY_ANNUAL_PREMIUM': POLICY_ANNUAL_PREMIUM
        }) as ATTRIBUTES
    FROM CUSTOMER_DATA
);

-- Wait for the search service to be ready
CALL SYSTEM$WAIT(30);

In [ ]:
-- Create a Search Service for Incident Types
CREATE OR REPLACE SEARCH SERVICE INCIDENT_TYPE_SEARCH
EXTERNAL_ACCESS_INTEGRATION = CORTEX_ACCESS
COMPUTE_POOL = CORTEX_COMPUTE
CONTENT = (
    SELECT DISTINCT
        INCIDENT_TYPE as CONTENT,
        TO_VARIANT({
            'INCIDENT_TYPE': INCIDENT_TYPE
        }) as ATTRIBUTES
    FROM CLAIM_DATA
);

-- Wait for the search service to be ready
CALL SYSTEM$WAIT(30);

In [ ]:
-- Let's test one of the search services
SELECT *
FROM TABLE(SEARCH(
    'COVERAGE_TYPE_SEARCH',
    'auto collision damage'
));

# Create Cortex Analyst Service

The Cortex Analyst service uses a semantic model to translate natural language questions into SQL. The semantic model is a YAML file that describes the structure and relationships of your data in business terms.

First, let's create and upload our insurance claims semantic model.

In [ ]:
!cat semantic_models/insurance_data.yaml

In [ ]:
-- Creating a stage to store the semantic model
CREATE OR REPLACE STAGE SEMANTIC_MODEL_STAGE;

In [ ]:
# Upload the semantic model YAML file to the stage
session.file.put('semantic_models/insurance_data.yaml', '@SEMANTIC_MODEL_STAGE/', auto_compress=False, overwrite=True)

In [ ]:
-- Check the files in the semantic model stage
LIST @SEMANTIC_MODEL_STAGE;

In [ ]:
-- Create a Cortex Analyst Service using the semantic model
CREATE OR REPLACE ANALYST SERVICE INSURANCE_CLAIMS_ANALYST
EXTERNAL_ACCESS_INTEGRATION = CORTEX_ACCESS
COMPUTE_POOL = CORTEX_COMPUTE
MODEL_LOCATION = '@SEMANTIC_MODEL_STAGE/insurance_data.yaml';

-- Wait for the analyst service to be ready
CALL SYSTEM$WAIT(30);

In [ ]:
-- List the analyst services
SHOW ANALYST SERVICES;

In [ ]:
-- Test the Cortex Analyst service with an example question
SELECT *
FROM TABLE(CORTEX_EXPLAIN('INSURANCE_CLAIMS_ANALYST', 'What is the average claim amount by incident type?'));

# Setup Cortex Agent

Now that we have set up our Cortex Search and Cortex Analyst services, let's create a Cortex Agent that can leverage both services to answer user questions about our insurance data.

Cortex Agents use a Claude model by default and can also use Mistral and Llama. They orchestrate multiple services to answer user questions, including both structured and unstructured data sources.

In [ ]:
-- Create a Cortex Agent that has access to our search and analyst services
CREATE OR REPLACE CORTEX AGENT INSURANCE_CLAIMS_AGENT
  SERVICE_NAMES = ('POLICY_DOCUMENTS_SEARCH', 'CLAIM_FORMS_SEARCH', 'COVERAGE_TYPE_SEARCH', 'POLICYHOLDER_SEARCH', 'INCIDENT_TYPE_SEARCH', 'INSURANCE_CLAIMS_ANALYST')
  EXTERNAL_ACCESS_INTEGRATION = CORTEX_ACCESS
  COMPUTE_POOL = CORTEX_COMPUTE
  PROMPT = $$
  You are an expert insurance claims analyst assistant. You have access to both structured insurance claims data and unstructured documents like policy information and claim forms.
  
  You should:
  - Provide accurate information from both structured data and documents
  - Be precise about policy information, deductibles, and claim processes
  - When analyzing fraud patterns, focus on statistical insights rather than making accusations
  - Maintain a professional, helpful tone
  - For complex questions, consider using both the data and documents to provide comprehensive answers
  $$;

-- Wait for the agent to be ready
CALL SYSTEM$WAIT(30);

In [ ]:
-- List the cortex agents
SHOW CORTEX AGENTS;

In [ ]:
-- Test the agent with a question about policy information
SELECT argument, response FROM TABLE(COMPLETE('INSURANCE_CLAIMS_AGENT', 'What are the different coverage types available and their descriptions?'));

In [ ]:
-- Test the agent with a question about claim data
SELECT argument, response FROM TABLE(COMPLETE('INSURANCE_CLAIMS_AGENT', 'What is the fraud rate for different incident types?'));

In [ ]:
-- Test the agent with a question that requires both policy documents and claim data
SELECT argument, response FROM TABLE(COMPLETE('INSURANCE_CLAIMS_AGENT', 'What is the deductible for comprehensive coverage and how does it affect claim settlements?'));

# Congratulations! 

You have successfully set up a Cortex Agent that can answer questions about insurance claims using both structured and unstructured data. This agent leverages:

1. **Cortex Search Services** for:
   - Policy documents and claim forms (unstructured data)
   - Coverage types, policyholders, and incident types (structured data literals)

2. **Cortex Analyst Service** with:
   - A semantic model for insurance claims data
   - SQL generation capabilities for complex analytical queries

3. **Cortex Agent** that:
   - Orchestrates all the services
   - Provides natural language responses to user questions
   - Combines insights from both structured and unstructured sources

You can now use the Streamlit app to interact with this agent and explore the insurance claims data in a more intuitive way.